# Load, Sample and Add english translation of messaged

In [17]:
import pandas as pd
import numpy as np
import json
# from deep_translator import GoogleTranslator

SAMPLE_SIZE = 100

# def translate(x):
#     try:
#         tranlsation = GoogleTranslator(source='auto', target='en').translate(x)
#     except:
#         tranlsation = ""
#     return tranlsation

def total_interactions(x):
    reactions_total = 0
    for _, value in json.loads(x.replace("'", '"')).items():
        reactions_total += value
    return reactions_total

def weighted_popularity_score(row):
    return np.mean([row.views, row.forwards, row.reaction_nb])

def min_max_normalization(col, df):
    return (df[col]-df[col].min())/(df[col].max()-df[col].min())


df = pd.read_csv('../../data/telegram/telegram.csv')
df['reaction_nb'] = df.reactions.apply(lambda x: total_interactions(x))

df["views_norm"] = min_max_normalization("views", df)
df["forwards_norm"] = min_max_normalization("forwards", df)
df["reaction_nb_norm"] = min_max_normalization("reaction_nb", df)

df['popularity_score'] = df.apply(lambda x: weighted_popularity_score(x), axis=1)
df = df.sort_values('popularity_score', ascending=False)

sampled = df.iloc[:SAMPLE_SIZE]
# sampled['englishMessageText'] = sampled.messageText.apply(lambda x: translate(x))
# sampled.to_csv('../../data/telegram/sample_with_translations.csv', index=False)

KeyboardInterrupt: 

In [26]:
df.messageDatetime.min()

'2016-09-06 06:56:10+00:00'

In [14]:
sampled[sampled.messageText.str.contains('Какой')]

,chat,messageDatetime,messageDate,messageText,views,forwards,replies,reactions,reaction_nb,views_norm,forwards_norm,reaction_nb_norm,popularity_score
363807,https://t.me/sashakots,2022-04-15 17:27:54+00:00,2022-04-15,Это хит!\n\n- Какой пароль на телефоне!\n\n- 1...,3937559,86490,0,"{'😁': 11245, '👍': 1165, '🔥': 466, '😱': 155, '👎...",13234,0.123714,0.317391,0.049559,1345761.0
323974,https://t.me/sashakots,2022-04-15 17:27:54+00:00,2022-04-15,Это хит!\n\n- Какой пароль на телефоне!\n\n- 1...,3937559,86490,0,"{'😁': 11245, '👍': 1165, '🔥': 466, '😱': 155, '👎...",13234,0.123714,0.317391,0.049559,1345761.0


In [16]:
sampled.chat.value_counts()

chat
https://t.me/medvedev_telegram       39
https://t.me/sashakots               18
https://t.me/readovkanews            11
https://t.me/rybar                    7
https://t.me/RKadyrov_95              7
https://t.me/vysokygovorit            6
https://t.me/neoficialniybezsonov     5
https://t.me/news_kremlin             2
https://t.me/epoddubny                2
https://t.me/RVvoenkor                2
https://t.me/wargonzo                 1
Name: count, dtype: int64

In [22]:
sampled.shape

(125, 14)

In [10]:
sampled[['chat', 'messageText']].to_dict()

{'chat': {414570: 'https://t.me/rybar',
  410078: 'https://t.me/rybar',
  409734: 'https://t.me/rybar',
  420006: 'https://t.me/rybar',
  31552: 'https://t.me/RKadyrov_95',
  415855: 'https://t.me/rybar',
  16242: 'https://t.me/medvedev_telegram',
  16582: 'https://t.me/medvedev_telegram',
  27778: 'https://t.me/news_kremlin',
  376484: 'https://t.me/sashakots',
  336651: 'https://t.me/sashakots',
  16628: 'https://t.me/medvedev_telegram',
  16545: 'https://t.me/medvedev_telegram',
  28635: 'https://t.me/news_kremlin',
  423911: 'https://t.me/rybar',
  760398: 'https://t.me/readovkanews',
  16581: 'https://t.me/medvedev_telegram',
  16584: 'https://t.me/medvedev_telegram',
  16282: 'https://t.me/medvedev_telegram',
  31596: 'https://t.me/RKadyrov_95',
  41053: 'https://t.me/neoficialniybezsonov',
  746934: 'https://t.me/readovkanews',
  751011: 'https://t.me/readovkanews',
  16544: 'https://t.me/medvedev_telegram',
  30744: 'https://t.me/RKadyrov_95',
  30993: 'https://t.me/RKadyrov_95

# Request propaganda detection for sample messages

In [24]:
import asyncio
import websockets
import json
import time
from tqdm import tqdm

async def connect_to_websocket(dataframe, client_id, message, progress_bar, semaphore):
    # Use a semaphore to limit the number of parallel connections
    async with semaphore:
        try:
            async with websockets.connect(
                "ws://13.48.71.178:8000/ws/analyze_propaganda", 
                ping_interval=None  # No timeout on ping
            ) as websocket:
                # Define the request data
                request_data = {
                    "model_name": "gpt-4o",  # Example model
                    "text": message,
                    "contextualize": "True"
                }

                # Send the request data
                await websocket.send(json.dumps(request_data))
                print(f"Client {client_id}: Request sent.")

                # Keep receiving messages until the server closes the connection
                while True:
                    try:
                        response = await websocket.recv()  # Wait for the next message from the server
                        print(f"Client {client_id}: Response received:\n{response}")
                        dataframe.at[client_id, "response"] = response  # Save response in dataframe
                    except websockets.ConnectionClosedOK:
                        # Server closed the connection gracefully
                        print(f"Client {client_id}: Connection closed OK.")
                        break
                    except websockets.ConnectionClosedError:
                        # Server closed with an error
                        print(f"Client {client_id}: Connection closed with an error.")
                        break
                    except Exception as e:
                        print(f"Client {client_id}: An error occurred: {e}")
                        break

        except Exception as e:
            print(f"Client {client_id}: An error occurred: {e}")
        finally:
            # Update the progress bar when a client request is complete
            progress_bar.update(1)

async def simulate_multiple_clients(dataframe, parallel_connections):
    tasks = []
    
    # Create a semaphore to limit parallel connections to 'parallel_connections'
    semaphore = asyncio.Semaphore(parallel_connections)
    
    # Create a tqdm progress bar for tracking finished tasks
    with tqdm(total=len(dataframe), desc="Finished Requests") as progress_bar:
        for i, row in enumerate(dataframe.itertuples()):
            tasks.append(connect_to_websocket(dataframe, i, row.messageText, progress_bar, semaphore))
        
        # Wait for all tasks to complete (i.e., wait for WebSocket closure for each client)
        await asyncio.gather(*tasks)

# Example dataframe
# Replace 'sampled' with your actual dataframe
parallel_connections = 5  # You can change this value to control how many requests run in parallel
await simulate_multiple_clients(sampled, parallel_connections)

# Save the dataframe after processing
sampled.to_csv('../../data/telegram/best_100_detection.csv', index=False)


Finished Requests:   1%|          | 1/125 [00:20<41:20, 20.01s/it]

Client 0: An error occurred: 
Client 1: An error occurred: 
Client 2: An error occurred: 
Client 3: An error occurred: 
Client 4: An error occurred: 


Finished Requests:   5%|▍         | 6/125 [00:40<11:50,  5.97s/it]

Client 5: An error occurred: 
Client 6: An error occurred: 
Client 7: An error occurred: 
Client 8: An error occurred: 
Client 9: An error occurred: 


Finished Requests:   9%|▉         | 11/125 [00:50<08:38,  4.55s/it]


CancelledError: 

In [3]:
df

,chat,messageDatetime,messageDate,messageText,views,forwards,replies,reactions,reaction_nb,views_norm,forwards_norm,reaction_nb_norm,popularity_score
414570,https://t.me/rybar,2023-08-15 15:51:47+00:00,2023-08-15,❓Вам нравится читать Рыбаря? \n\nНравится полу...,31828037,2708,0,{},0,1.000000,0.009938,0.0,1.061025e+07
410078,https://t.me/rybar,2023-03-27 17:25:15+00:00,2023-03-27,❗️ СДАЙ ТРОФЕЙ — ПОМОГИ РОССИЙСКОМУ ВПК!\n\nКо...,23186885,6872,0,{},0,0.728505,0.025218,0.0,7.731252e+06
409734,https://t.me/rybar,2023-03-16 04:13:46+00:00,2023-03-16,❗️Поддержите команду Рыбаря\n\nНаш проект фина...,16153159,1684,0,{},0,0.507514,0.006180,0.0,5.384948e+06
420006,https://t.me/rybar,2024-02-19 23:00:41+00:00,2024-02-19,📄📌 Зачем нужен Рыбарь+?\n\nНа нашем платном за...,11460978,842,0,{},0,0.360091,0.003090,0.0,3.820607e+06
31552,https://t.me/RKadyrov_95,2022-10-01 13:49:59+00:00,2022-10-01,Я всегда говорил: нет ничего лучше озвученной ...,9193577,272503,0,"{'👍': 228441, '👏': 15755, '🔥': 10721, '❤': 904...",267035,0.288852,1.000000,1.0,3.244372e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
645290,https://t.me/RVvoenkor,2022-03-14 16:22:42+00:00,2022-03-14,This message couldn't be displayed on your dev...,0,0,0,{},0,0.000000,0.000000,0.0,0.000000e+00
645291,https://t.me/RVvoenkor,2022-03-14 16:22:42+00:00,2022-03-14,This message couldn't be displayed on your dev...,0,0,0,{},0,0.000000,0.000000,0.0,0.000000e+00
645292,https://t.me/RVvoenkor,2022-03-14 16:22:42+00:00,2022-03-14,This message couldn't be displayed on your dev...,0,0,0,{},0,0.000000,0.000000,0.0,0.000000e+00
644509,https://t.me/RVvoenkor,2022-03-06 12:19:16+00:00,2022-03-06,This message couldn't be displayed on your dev...,0,0,0,{},0,0.000000,0.000000,0.0,0.000000e+00


In [11]:
def unfold_results(x):
    obj = json.loads(x['response'])
    for key, value in obj['data'].items():
        x[key] = 1

    return x

new = sampled.apply(lambda x: unfold_results(x), axis=1)
new.fillna(0, inplace=True)
new['is_propaganda'] = new[['Appeal_to_Authority', 'Appeal_to_fear-prejudice',
       'Bandwagon, Reductio_ad_hitlerum', 'Black-and-White_Fallacy',
       'Causal_Oversimplification', 'Doubt', 'Exaggeration, Minimization',
       'Flag-Waving', 'Loaded_Language', 'Name_Calling, Labeling',
       'Repetition', 'Slogans', 'Whataboutism, Straw_Men, Red_Herring']].sum(axis=1)
new['is_propaganda'] = new['is_propaganda']>0
print(new['is_propaganda'].value_counts())
new.to_csv('../../data/telegram/unfolded.csv')

is_propaganda
True     87
False    13
Name: count, dtype: int64

# Telethon

In [4]:
# Import libraries
import os
from dotenv import load_dotenv
from telethon import TelegramClient
from telethon.errors import SessionPasswordNeededError
from tqdm.notebook import tqdm
import getpass
from IPython.display import clear_output
import asyncio
from telethon.tl.types import Message
import inspect

# Option 1: Load environment variables from .env file
load_dotenv()
TELEGRAM_API_ID = os.getenv("TELEGRAM_API_ID")
TELEGRAM_API_HASH = os.getenv("TELEGRAM_API_HASH")

# Option 2: Directly set API credentials (Uncomment if not using .env)
# TELEGRAM_API_ID = 'your_api_id'
# TELEGRAM_API_HASH = 'your_api_hash'

# Initialize the Telegram client
client = TelegramClient('simple_session', TELEGRAM_API_ID, TELEGRAM_API_HASH)

# Authentication function
async def authenticate_client():
    await client.connect()
    
    if not await client.is_user_authorized():
        phone = input("Enter your phone number (with country code, e.g., +123456789): ")
        await client.send_code_request(phone)
        code = input("Enter the code you received: ")
        try:
            await client.sign_in(phone, code)
        except SessionPasswordNeededError:
            password = getpass.getpass("Two-step verification enabled. Enter your password: ")
            await client.sign_in(password=password)
        clear_output()
        print("Authentication successful!")
    else:
        print("Client is already authorized.")

# Run authentication
await authenticate_client()

# Function to print message attributes
def print_message_attributes(chat, limit=10):
    """
    Fetches and prints all attributes of the first `limit` messages from a specified chat.

    :param chat: The username or ID of the Telegram chat to scrape.
    :param limit: Number of messages to retrieve.
    """
    async def fetch_messages():
        print(f'\nScraping the first {limit} messages from chat: {chat}\n')
        try:
            # Fetch messages asynchronously
            messages = await client.iter_messages(chat, limit=limit).to_list()
            
            for idx, message in enumerate(messages, start=1):
                print(f"--- Message {idx} ---")
                # Get all attributes of the message
                attributes = {attr: getattr(message, attr) for attr in dir(message) 
                              if not attr.startswith('_') and not inspect.ismethod(getattr(message, attr))}
                for key, value in attributes.items():
                    print(f"{key}: {value}")
                print("\n")
        except Exception as e:
            print(f"An error occurred: {e}")

    # Run the asynchronous fetch
    asyncio.run(fetch_messages())

# Specify the chat to scrape
chat_to_scrape = 'https://t.me/opersvodki'  # Replace with your target chat https://t.me/medvedev_telegram

# Fetch and print message attributes
print_message_attributes(chat_to_scrape, limit=10)


PhoneNumberInvalidError: The phone number is invalid (caused by SendCodeRequest)

In [37]:
from telethon.sync import TelegramClient
import os

# Your API ID and API Hash from my.telegram.org
TELEGRAM_API_ID = os.getenv("TELEGRAM_API_ID")
TELEGRAM_API_HASH = os.getenv("TELEGRAM_API_HASH")
# Peer ID of the Telegram channel
peer_id = '1315735637'

# Connect to the Telegram client
client = TelegramClient('session_name', TELEGRAM_API_ID, TELEGRAM_API_HASH)

entity = client.get_entity(peer_id)

    # Print the channel name
print('Channel Name:', entity.title)


AttributeError: 'coroutine' object has no attribute 'title'

In [40]:
import os
import asyncio
from telethon import TelegramClient
from telethon.tl.types import PeerChannel

# Fetch API credentials from environment variables and convert API_ID to integer
TELEGRAM_API_ID = int(os.getenv("TELEGRAM_API_ID"))
TELEGRAM_API_HASH = os.getenv("TELEGRAM_API_HASH")

# Peer ID of the Telegram channel (ensure it's an integer)
peer_id = 1315735637  # Remove quotes to make it an integer

async def main():
    # Initialize the Telegram client with a unique session name
    async with TelegramClient('unique_session_name', TELEGRAM_API_ID, TELEGRAM_API_HASH) as client:
        try:
            # Retrieve the entity using the peer ID
            entity = await client.get_entity(PeerChannel(peer_id))
            
            # Print the channel name
            print('Channel Name:', entity.title)
        except ValueError:
            print('Invalid peer ID or entity not found')
        except Exception as e:
            print(f'An error occurred: {e}')

# Run the asynchronous main function
if __name__ == "__main__":
    asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop

In [41]:
import os
import asyncio
import logging
from telethon import TelegramClient
from telethon.tl.types import PeerChannel

# Configure logging
logging.basicConfig(
    level=logging.INFO,  # Set to DEBUG for more detailed logs
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Fetch API credentials from environment variables and convert API_ID to integer
TELEGRAM_API_ID = os.getenv("TELEGRAM_API_ID")
TELEGRAM_API_HASH = os.getenv("TELEGRAM_API_HASH")

# Validate API credentials
if not TELEGRAM_API_ID or not TELEGRAM_API_HASH:
    logger.error("TELEGRAM_API_ID and TELEGRAM_API_HASH must be set as environment variables.")
    exit(1)

try:
    TELEGRAM_API_ID = int(TELEGRAM_API_ID)
except ValueError:
    logger.error("TELEGRAM_API_ID must be an integer.")
    exit(1)

# Peer ID of the Telegram channel (ensure it's an integer)
peer_id = 1315735637  # Ensure this is the correct integer ID

async def main():
    # Initialize the Telegram client with a unique session name
    try:
        async with TelegramClient('unique_session_name', TELEGRAM_API_ID, TELEGRAM_API_HASH) as client:
            logger.info("Client started successfully.")

            try:
                # Retrieve the entity using the peer ID
                entity = await client.get_entity(PeerChannel(peer_id))
                
                # Print the channel name
                print('Channel Name:', entity.title)
                logger.info(f'Channel Name: {entity.title}')
            except ValueError:
                logger.error('Invalid peer ID or entity not found.')
            except Exception as e:
                logger.exception(f'An unexpected error occurred: {e}')

    except Exception as e:
        logger.exception(f'Failed to initialize TelegramClient: {e}')

# Run the asynchronous main function
if __name__ == "__main__":
    asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop

In [51]:
async with TelegramClient('SessionName', TELEGRAM_API_ID, TELEGRAM_API_HASH) as client:
    # Fetch the entity using the peer ID
    entity = await client.get_entity(PeerChannel(1391419522))
    
    # Print the channel name
    print('Channel Name:', entity.title)

2024-10-17 16:44:19,902 - telethon.network.mtprotosender - INFO - Connecting to 149.154.167.91:443/TcpFull...
2024-10-17 16:44:19,924 - telethon.network.mtprotosender - INFO - Connection to 149.154.167.91:443/TcpFull complete!
2024-10-17 16:44:20,155 - telethon.network.mtprotosender - INFO - Disconnecting from 149.154.167.91:443/TcpFull...
2024-10-17 16:44:20,157 - telethon.network.mtprotosender - INFO - Disconnection from 149.154.167.91:443/TcpFull complete!


ValueError: Could not find the input entity for PeerChannel(channel_id=1391419522) (PeerChannel). Please read https://docs.telethon.dev/en/stable/concepts/entities.html to find out more details.

In [48]:
PeerChannel(1315735637)

# Analyse network between channels

In [69]:
import pandas as pd

df = pd.read_csv('../../data/telegram/messages_scraped.csv')
print("size df total", df.shape)
# Convert 'messageDate' column to datetime format
df['messageDate'] = pd.to_datetime(df['messageDate'])

# Filter rows with 'messageDate' after 2022
df = df[df['messageDate'] > '2022-01-01']
print("size df after 2022", df.shape)

/var/folders/jx/2l7p1chd2l7djbz_8s3s_4z80000gn/T/ipykernel_59738/1289742480.py:3: DtypeWarning: Columns (24,25,27,28,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../data/telegram/messages_scraped.csv')


size df total (1623145, 39)
size df after 2022 (1275438, 39)


In [68]:
df.shape, filtered_df.shape

((1623145, 39), (1275438, 39))

In [55]:
# Select the relevant columns
df_selected = df[['peer_id', 'chat']]

# Drop duplicate peer_id entries to ensure each key is unique
df_unique = df_selected.drop_duplicates(subset='peer_id')

# Set 'peer_id' as the index and convert the 'chat' column to a dictionary
peer_id_to_chat = df_unique.set_index('peer_id')['chat'].to_dict()

In [66]:
df.head()

,chat,id,peer_id,messageDatetime,messageDate,messageText,out,mentioned,media_unread,silent,...,grouped_id,restriction_reason,ttl_period,quick_reply_shortcut_id,effect,factcheck,views,forwards,replies,reactions
0,https://t.me/soldatskayapravda,2,1719098803,2022-04-27 15:19:56+00:00,2022-04-27,!,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,10954,2,0,"{'👍': 55, '🤮': 4, '🔥': 1}"
1,https://t.me/medvedev_telegram,4,1572748754,2022-03-17 06:59:14+00:00,2022-03-17,Это мой первый пост в Telegram. Обстоятельства...,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,1424178,12565,0,{}
2,https://t.me/medvedev_telegram,5,1572748754,2022-03-18 07:00:10+00:00,2022-03-18,Несколько слов о социальных сетях.\n\nДля милл...,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,719086,2426,0,{}
3,https://t.me/medvedev_telegram,6,1572748754,2022-03-19 12:21:53+00:00,2022-03-19,1. В последнее время одна из самых обсуждаемых...,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,359106,1023,0,{}
4,https://t.me/medvedev_telegram,7,1572748754,2022-03-19 12:22:14+00:00,2022-03-19,"2. Попалась на глаза ироничная картинка, котор...",False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,331754,830,0,{}


In [70]:
import pandas as pd
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the number of messages to observe
# MESSAGE_OBSERVED = 10000

# Step 1: Create the peer_id_to_chat dictionary
peer_id_to_chat = df[['peer_id', 'chat']].drop_duplicates().set_index('peer_id')['chat'].to_dict()

# Step 2: Assign unique colors to each unique chat
unique_chats = set(peer_id_to_chat.values())
# Do not discard "unknown name" to handle it separately

# Choose a colormap
cmap = plt.get_cmap('tab20')  # 'tab20' has 20 distinct colors
num_colors = len(unique_chats)

# If number of unique chats exceeds the colormap, use another colormap or extend the existing one
if num_colors > cmap.N:
    cmap = plt.get_cmap('hsv')  # 'hsv' can provide a wide range of colors
    colors = [mcolors.rgb2hex(cmap(i / num_colors)) for i in range(num_colors)]
else:
    colors = [mcolors.rgb2hex(cmap(i)) for i in range(num_colors)]

# Create a dictionary mapping chat names to colors
chat_to_color = {chat: colors[i] for i, chat in enumerate(unique_chats)}

# Assign a default color for nodes with unknown names
default_node_color = '#C0C0C0'  # Silver

# Step 3: Filter the DataFrame to include only the top MESSAGE_OBSERVED messages by 'forwards'
df_max_forwards = df.sort_values('forwards', ascending=False)#.iloc[:MESSAGE_OBSERVED]

# Step 4: Further filter to include only forwarded messages
forwarded_messages = df_max_forwards[df_max_forwards['fwd_from'].notna()]

# Step 5: Initialize a directed graph
G = nx.DiGraph()

# Step 6: Iterate over each forwarded message to add edges to the graph
for index, row in forwarded_messages.iterrows():
    source_peer_id = row['fwd_from']
    target_peer_id = row['peer_id']
    message_id = row['id']
    message_text = row['messageText'] if pd.notna(row['messageText']) else ''
    message_date = row['messageDate']
    
    # Get chat names or use 'unknown name'
    source_chat = peer_id_to_chat.get(source_peer_id, "unknown name")
    target_chat = peer_id_to_chat.get(target_peer_id, "unknown name")
    
    # Use peer_id as unique identifier (converted to string)
    source_node = str(source_peer_id)
    target_node = str(target_peer_id)
    
    # Determine node colors based on chat names
    source_color = chat_to_color.get(source_chat, default_node_color)
    target_color = chat_to_color.get(target_chat, default_node_color)
    
    # Add nodes with label as chat names and assigned colors
    if not G.has_node(source_node):
        G.add_node(source_node, label=source_chat, color=source_color)
    if not G.has_node(target_node):
        G.add_node(target_node, label=target_chat, color=target_color)
    
    # Add an edge from source to target with message details as edge attributes
    G.add_edge(
        source_node,
        target_node,
        message_id=message_id,
        text=message_text,
        date=message_date
    )

# Step 7: Create a PyVis Network
net = Network(height='750px', width='100%', notebook=True, directed=True)

# Customize the physics layout (optional for better visualization)
net.force_atlas_2based()

# Add nodes with labels and colors
for node, data in G.nodes(data=True):
    label = data.get('label', 'unknown name')
    color = data.get('color', default_node_color)
    net.add_node(
        node,
        label=label,
        title=label,  # Tooltip on hover
        color=color,  # Node color based on chat
        size=15
    )

# Add edges with tooltips
for source, target, data in G.edges(data=True):
    message_id = data.get('message_id', '')
    text = data.get('text', '')
    date = data.get('date', '')
    tooltip = f"<b>Message ID:</b> {message_id}<br><b>Date:</b> {date}<br><b>Text:</b> {text}"
    
    net.add_edge(
        source,
        target,
        title=tooltip  # Tooltip on hover
        # No color assigned for edges
    )

# Optionally, enable additional features like showing physics controls
net.show_buttons(filter_=['physics'])

# Generate and save the interactive graph to an HTML file
net.show(f'interactive_graph.html')


interactive_graph.html


In [58]:
import pandas as pd
import networkx as nx
from pyvis.network import Network

# Assume 'df' is your DataFrame
# Filter the DataFrame to include only forwarded messages and sort by max forwards
df_10000_max_forwards = df.sort_values('forwards', ascending=False).iloc[:10000]
forwarded_messages = df_10000_max_forwards[df_10000_max_forwards['fwd_from'].notna()]

# Initialize a directed graph
G = nx.DiGraph()

# Iterate over each forwarded message to add edges to the graph
for index, row in forwarded_messages.iterrows():
    source = str(row['fwd_from'])   # Convert to string
    target = str(row['peer_id'])    # Convert to string
    message_id = row['id']
    message_text = row['messageText']
    message_date = row['messageDate']
    
    # Add nodes for source and target
    G.add_node(source)
    G.add_node(target)
    
    # Add an edge from source to target with message details as edge attributes
    G.add_edge(source, target, message_id=message_id, text=message_text, date=message_date)

# Create a PyVis Network
net = Network(height='750px', width='100%', notebook=True, directed=True)

# Add nodes and edges from your NetworkX graph
net.from_nx(G)

# Add tooltips to edges to display message details on hover
for edge in net.edges:
    message_id = edge['message_id']
    text = edge['text']
    date = edge['date']
    edge['title'] = f"<b>Message ID:</b> {message_id}<br><b>Date:</b> {date}<br><b>Text:</b> {text}"

# Customize the physics for better visualization (optional)
net.show_buttons(filter_=['physics'])

# Generate and display the interactive graph
net.show('interactive_graph.html')


interactive_graph.html


In [ ]:
import pandas as pd
import networkx as nx
from pyvis.network import Network

# Filter the DataFrame to include only forwarded messages
forwarded_messages = df[df['fwd_from'].notna()]

# Initialize a directed graph
G = nx.DiGraph()

# Iterate over each forwarded message to add edges to the graph
for index, row in forwarded_messages.iterrows():
    source = row['fwd_from']
    target = row['peer_id']
    message_id = row['id']
    message_text = row['messageText']
    message_date = row['messageDate']

    # Add nodes with labels
    G.add_node(source, label=str(source))
    G.add_node(target, label=str(target))

    # Add an edge from source to target with message details as edge attributes
    G.add_edge(
        source,
        target,
        message_id=message_id,
        text=message_text,
        date=str(message_date)[:10],  # Convert date to string
    )


import pandas as pd
import networkx as nx

# Assume 'df' is your DataFrame
# Filter the DataFrame to include only forwarded messages
# sort by max forwards
df_10000_max_forwards = df.sort_values('forwards', ascending=False).iloc[:10000]
forwarded_messages = df_10000_max_forwards[df_10000_max_forwards['fwd_from'].notna()]

# Initialize a directed graph
G = nx.DiGraph()

# Iterate over each forwarded message to add edges to the graph
for index, row in forwarded_messages.iterrows():
    source = row['fwd_from']
    target = row['peer_id']
    message_id = row['id']
    message_text = row['messageText']
    message_date = row['messageDate']
    
    # Add nodes for source and target if they don't exist
    if not G.has_node(source):
        G.add_node(source)
    if not G.has_node(target):
        G.add_node(target)
    
    # Add an edge from source to target with message details as edge attributes
    G.add_edge(source, target, message_id=message_id, text=message_text, date=message_date)

import matplotlib.pyplot as plt

# Draw the graph
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, node_size=500, node_color='lightblue', arrowsize=20)
plt.show()



In [59]:
df[df.peer_id==1315735637]
# df[df.peer_id==1093357968]

,chat,id,peer_id,messageDatetime,messageDate,messageText,out,mentioned,media_unread,silent,...,grouped_id,restriction_reason,ttl_period,quick_reply_shortcut_id,effect,factcheck,views,forwards,replies,reactions
822723,https://t.me/SolovievLive,4,1315735637,2019-06-09 17:09:41+00:00,2019-06-09,"Если бы этого дела не было, оппозиции надо был...",False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,91837,414,0,"{'👍': 326, '👎': 23, '🔥': 6, '👏': 4, '🤔': 4, '😢..."
822724,https://t.me/SolovievLive,5,1315735637,2019-06-09 17:09:41+00:00,2019-06-09,Крайне чувствительной для определённой категор...,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,71668,141,0,"{'👍': 119, '👎': 10, '🔥': 4, '👏': 2, '😢': 2}"
822725,https://t.me/SolovievLive,6,1315735637,2019-06-10 20:16:03+00:00,2019-06-10,Сегодня все Голунов. И завтра. И ещё несколько...,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,21919,40,0,"{'👍': 108, '👎': 7, '🔥': 2, '😢': 2}"
822726,https://t.me/SolovievLive,7,1315735637,2019-06-10 20:16:44+00:00,2019-06-10,А дальше что? \n\nСценарий первый: Голунов на ...,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,39034,118,0,"{'👍': 64, '👎': 5, '🤔': 3}"
822727,https://t.me/SolovievLive,9,1315735637,2019-06-11 07:54:29+00:00,2019-06-11,Самое зубодробительное в нашем Уголовном кодек...,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,266,0,0,"{'👍': 184, '👎': 12, '👏': 3}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1055390,https://t.me/SolovievLive,289098,1315735637,2024-10-16 13:08:53+00:00,2024-10-16,🖥️ Спикер Верховной Рады Стефанчук показал жур...,False,False,False,False,...,1.383267e+16,NaN,NaN,NaN,NaN,NaN,53363,36,0,"{'🤔': 620, '👎': 171, '👍': 94, '👏': 17, '🔥': 15..."
1055391,https://t.me/SolovievLive,289100,1315735637,2024-10-16 13:17:05+00:00,2024-10-16,🖥️ В Подмосковье неизвестный в упор расстрелял...,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,47111,172,0,"{'😢': 1614, '👎': 218, '🤔': 155, '👍': 81, '🔥': ..."
1055392,https://t.me/SolovievLive,289101,1315735637,2024-10-16 13:24:55+00:00,2024-10-16,"Генерал Иван Попов заявил в суде, что не призн...",False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,37681,35,0,"{'👍': 1515, '🤔': 213, '👏': 79, '👎': 47, '🔥': 2..."
1055393,https://t.me/SolovievLive,289102,1315735637,2024-10-16 13:30:16+00:00,2024-10-16,Associated Press: Зеленский в сентябре дал сою...,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,32455,0,0,"{'👍': 387, '🤔': 127, '👎': 37, '👏': 12, '🔥': 9}"
